Adapted from this gist: https://gist.github.com/aodag/ed327394bd2cfabda0c337eacce1ab69  
Using the new attrs API breaks it though

In [1]:
from typing import Optional
import attrs
from sqlalchemy import (
    Table,
    MetaData,
    Column,
    Unicode,
    Integer,
    create_engine,
    ForeignKey,
)
from sqlalchemy.orm import registry, sessionmaker, relationship

In [2]:
@attrs.define
class Job:
    id: Optional[int] = attrs.field(default=None)
    name: str = attrs.field(default="")

In [3]:
@attrs.define
class Person:
    id: Optional[int] = attrs.field(default=None)
    name: str = attrs.field(default="")
    job: Job = attrs.field(default=None)

In [4]:
mapper_registry = registry()


In [5]:
person_table = Table(
    "person",
    mapper_registry.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", Unicode),
    Column("job_id", Integer, ForeignKey("job.id")),
)

In [6]:
job_table = Table(
    "job", 
    mapper_registry.metadata,
    Column("id", Integer, primary_key=True), 
    Column("name", Unicode)
)

In [7]:
engine = create_engine("sqlite:///")
engine.echo = True
Session = sessionmaker(bind=engine)


In [8]:
mapper_registry.map_imperatively(Job, job_table)
mapper_registry.map_imperatively(Person, person_table, properties={"job": relationship(Job)})
mapper_registry.metadata.create_all(bind=engine)

ArgumentError: Mapper mapped class Job->job could not assemble any primary key columns for mapped table 'job'

In [9]:
j = Job(name="engineer")
p = Person(name="aodag", job=j)

print(p)
session = Session()
session.add(p)
session.flush()
print(p)
print([attr.asdict(p) for p in session.query(Person).all()])
# -> [{'id': 1, 'name': 'aodag', 'job': {'id': 1, 'name': 'engineer'}}]


AttributeError: 'Job' object has no attribute '_sa_instance_state'